In [1]:
import os
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from selenium.webdriver.common.keys import Keys # Importe les clefs pour les touches du clavier
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import math
import time
from time import sleep 

from tqdm import tqdm # librairie permettant d'indiquer le pourcentage de la boucle for


import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
gaming = pd.read_csv('vgsales.csv', index_col=0)
gaming.head(5)


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
gaming_parts = np.array_split(gaming, 16)
gaming_parts[7].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
7267,Dead Island,PC,2011.0,Action,Deep Silver,0.12,0.07,0.00,0.03,0.22
7268,Growlanser: Heritage of War,PS2,2006.0,Role-Playing,Rising Star Games,0.11,0.08,0.00,0.03,0.22
7269,Hasbro Family Game Night,DS,2009.0,Puzzle,Electronic Arts,0.01,0.19,0.00,0.02,0.22
7270,Shin Megami Tensei x Fire Emblem,WiiU,2015.0,Role-Playing,Nintendo,0.10,0.04,0.06,0.01,0.22
7271,Rugby 2004,PS2,2003.0,Sports,Electronic Arts,0.11,0.08,0.00,0.03,0.22


In [4]:
def scroll(value): #Définition d'une fonction pour scroller automatiquement
    for i in range(20): #nombre de micro-scroll à effectuer
        driver.execute_script("window.scrollBy(0, {})".format(value))
        time.sleep(0.1) #Temps entre chaque scroll

In [5]:
gaming_JVC=gaming_parts[7]
gaming_JVC=gaming_JVC.iloc[192:]

In [6]:
#scrapping part 8
#gaming_JVC=gaming_parts[7]

# Ouvrir une page Google Chrome
driver = webdriver.Chrome(ChromeDriverManager().install()) 
# créer une attente implicite
driver.implicitly_wait(10)

#Lancer une recherche en utilisant un fichier excel ou csv (en ittérant sur une colonne)

for i, row in tqdm(gaming_JVC.iterrows(), total = len(gaming_JVC)):
    
    # Ouvrir l'adresse du site web
    driver.get('https://www.jeuxvideo.com/tous-les-jeux/')
    try:
        # accepter les cookies
        driver.find_element(By.XPATH, "//*[@id='cmp-main']/button[2]").click()
    except:
        pass
    
    #Ajout d'un temps de deux secondes avant de lancer l'action suivante
    time.sleep(1) 
    
    loupe_recherche=driver.find_element(By.CLASS_NAME, 'searchContainer__1f9uG_')
    loupe_recherche.click()

    try:
        barre_recherche_site=driver.find_element(By.CLASS_NAME,"searchInput__38b75-")
        barre_recherche_site.send_keys(row['Name'])
        barre_recherche_site.send_keys(Keys.ENTER)

    except:
        pass

    try:    
        premier_jeu=driver.find_elements(By.CLASS_NAME,"col")[0]
        premier_jeu.click()

        # cliquer sur la bonne plateforme
        boutons_plateforme=driver.find_elements(By.CLASS_NAME,"gameHeaderBanner__platform")
        boutons_plateforme
    except:
        pass

    try:
        for bouton in boutons_plateforme:
            if str(bouton.text) == str(row['Platform']):
                bouton.click()
            else:
                pass
    except:
        pass

    try:
        test_note=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__gauge")
        gaming_JVC.loc[i, 'test_note_JVC']=test_note.text
    except:
        gaming_JVC.loc[i, 'test_note_JVC']='no_data'
    
    try:
        avis_count=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__count")
        gaming_JVC.loc[i, 'avis_count_JVC']=avis_count.text
    except:
        gaming_JVC.loc[i, 'avis_count_JVC']='no_data'

    try:
        avis_note=driver.find_elements(By.CLASS_NAME, "gameCharacteristicsMain__gauge")[1]
        gaming_JVC.loc[i, 'avis_note_JVC']=avis_note.text
    except:
        gaming_JVC.loc[i, 'avis_note_JVC']='no_data'
    
    scroll(100)

    try:
        game_caracteristique=driver.find_element(By.CLASS_NAME, "gameCharacteristicsDetailed__table")
        gaming_JVC.loc[i, 'description_JVC']=game_caracteristique.text
    except:
        gaming_JVC.loc[i, 'description_JVC']='no_data'
        


  6%|▌         | 49/845 [28:44<7:29:31, 33.88s/it] 

In [ ]:
driver.close()

In [7]:
gaming_JVC.head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,test_note_JVC,avis_count_JVC,avis_note_JVC,description_JVC
Rank,,,,,,,,,,,,,,
7267,Dead Island,PC,2011.0,Action,Deep Silver,0.12,0.07,0.00,0.03,0.22,16,(497),15.3,Editeur(s) / Développeur(s) Deep Silver Techla...
7268,Growlanser: Heritage of War,PS2,2006.0,Role-Playing,Rising Star Games,0.11,0.08,0.00,0.03,0.22,no_data,no_data,no_data,no_data
7269,Hasbro Family Game Night,DS,2009.0,Puzzle,Electronic Arts,0.01,0.19,0.00,0.02,0.22,--,no_data,no_data,Editeur(s) / Développeur(s) Electronic Arts\nS...
7270,Shin Megami Tensei x Fire Emblem,WiiU,2015.0,Role-Playing,Nintendo,0.10,0.04,0.06,0.01,0.22,no_data,no_data,no_data,no_data
7271,Rugby 2004,PS2,2003.0,Sports,Electronic Arts,0.11,0.08,0.00,0.03,0.22,no_data,no_data,no_data,no_data
7272,Disney's Magical Quest Starring Mickey and Minnie,GBA,2002.0,Action,Nintendo,0.16,0.06,0.00,0.00,0.22,no_data,no_data,no_data,no_data
7273,Spirit Camera: The Cursed Memoir,3DS,2012.0,Adventure,Nintendo,0.12,0.02,0.06,0.01,0.22,14,(42),13.3,Editeur(s) / Développeur(s) Tecmo Koei Nintend...
7274,NBA 06,PSP,2005.0,Sports,Sony Computer Entertainment,0.20,0.00,0.00,0.02,0.22,13,(25),17.8,Editeur(s) / Développeur(s) 2K Sports Visual C...
7275,Naruto: Ninja Council,GBA,2003.0,Action,Tomy Corporation,0.16,0.06,0.00,0.00,0.22,8,(27),11.3,Editeur(s) / Développeur(s) Takara Tomy 505 Ga...


In [8]:
gaming_JVC.to_csv('gaming_JVC_8bis.csv')